In [23]:
MSASL_TRAIN_PATH = "./MS-ASL/MSASL_train.json"
MSASL_VAL_PATH = "./MS-ASL/MSASL_val.json"
MSASL_TEST_PATH = "./MS-ASL/MSASL_test.json"
MSASL_CLASSES_PATH = "./MS-ASL/MSASL_classes.json"

In [24]:
import json
import pandas as pd

with open(MSASL_TRAIN_PATH) as f:
    train_data = json.load(f)

with open(MSASL_VAL_PATH) as f:
    val_data = json.load(f)

with open(MSASL_TEST_PATH) as f:
    test_data = json.load(f)
    
with open(MSASL_CLASSES_PATH) as f:
    class_data = json.load(f)

train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)
class_df = pd.DataFrame(class_data)

class_df.columns = ['label_name']
class_df['label'] = class_df.index

train_df = train_df.merge(class_df, how='left', on='label')
val_df = val_df.merge(class_df, how='left', on='label')
test_df = test_df.merge(class_df, how='left', on='label')

In [25]:
train_df

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review,label_name
0,match [light-a-MATCH],match,0.000,0,0,0,83,match light-a-MATCH,830,360.0,30.000,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.05754461884498596, 0.21637457609176636, 1.0...",640.0,NaN,match
1,FAIL,fail,0.000,0,-1,0,74,FAIL,542,360.0,25.000,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.0657794177532196, 0.16717177629470825, 0.93...",480.0,NaN,fail
2,laugh,laugh,0.000,4,26,0,31,SignSchool Laugh with Legs 2,312,360.0,29.970,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.13188594579696655, 0.32334136962890625, 1.0...",640.0,NaN,laugh
3,BOOK,book,0.000,0,-1,0,66,BOOK(3),38,360.0,25.000,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.05569887161254883, 0.25173279643058777, 0.9...",480.0,NaN,book
4,sign-language,sign language,0.000,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360.0,29.970,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.03904399275779724, 0.24198183417320251, 1.0...",640.0,NaN,sign language
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16049,WRITE,write,539.939,32,17,16182,16301,ASL 1 Unit 1 Vocabulary,53,360.0,29.970,543.910,https://www.youtube.com/watch?v=fNg_sJ9f8EI,write,"[0.03217703104019165, 0.3161315321922302, 1.0,...",640.0,NaN,write
16050,hot,hot,0.000,2,42,0,127,Hot,135,360.0,29.657,4.282,https://www.youtube.com/watch?v=MkXUHhsMAns,hot,"[0.00036713480949401855, 0.20649650692939758, ...",640.0,NaN,hot
16051,hi,hi,0.000,0,0,0,47,Hi,379,360.0,29.970,1.568,www.youtube.com/watch?v=rhfJGeMDMzQ,hi,"[0.013705313205718994, 0.17314371466636658, 1....",640.0,NaN,hi
16052,Learn,learn,0.000,135,-1,0,104,ASL Learn,22,360.0,30.000,3.467,https://www.youtube.com/watch?v=n-b2NMAwk28,learn,"[0.18269836902618408, 0.19255371391773224, 1.0...",480.0,NaN,learn


In [21]:
import os
import pandas as pd
from moviepy.video.io.VideoFileClip import VideoFileClip
import yt_dlp
import glob

OUTPUT_DIR = "msasl_clips"
NUMBER_OF_CLIPS = 10

os.makedirs(OUTPUT_DIR, exist_ok=True)

df = train_df  
df = df[["url", "start_time", "end_time", "label", "label_name", "file"]]

def download_and_clip(row):
    video_url = row["url"]
    start = float(row["start_time"])
    end = float(row["end_time"])
    label = row["label_name"]
    filename_base = row["file"].replace(" ", "_")

    label_dir = os.path.join(OUTPUT_DIR, label)
    os.makedirs(label_dir, exist_ok=True)

    full_video_path_no_ext = os.path.join(label_dir, f"{filename_base}_full")
    clipped_path = os.path.join(label_dir, f"{filename_base}_clip.mp4")

    if os.path.exists(clipped_path):
        return

    ydl_opts = {
        "format": "bestvideo+bestaudio/best",
        "merge_output_format": "mp4",  
        "outtmpl": full_video_path_no_ext,  
        "quiet": True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
            
    except Exception as e:
        print(f"Failed to download {video_url}: {e}")
        return

    downloaded_files = glob.glob(full_video_path_no_ext + "*")
    
    if downloaded_files:
        downloaded_path = downloaded_files[0]  
        
    else:
        print(f"Failed to find downloaded file for {video_url}")
        return

    try:
        with VideoFileClip(downloaded_path) as video:
            clip = video.subclip(start, end)
            clip.write_videofile(clipped_path, codec="libx264", audio=False, verbose=False, logger=None)
            
    except Exception as e:
        print(f"Failed to clip {downloaded_path}: {e}")
        return

    os.remove(downloaded_path)
    
for _, row in df.head(NUMBER_OF_CLIPS).iterrows():  
    download_and_clip(row)

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 1202, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 1, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 2.88, 'bitrate': 1193, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1920, 1080], 'video_bitrate': 1202, 'video_fps': 30.0, 'default

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 480], 'bitrate': 299, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 1, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 3.08, 'bitrate': 303, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Main)', 'video_size': [640, 480], 'video_bitrate': 299, 'video_fps': 25.0, 'default_audio_

ERROR: [youtube] 9FdHlMOnVjg: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Failed to download www.youtube.com/watch?v=9FdHlMOnVjg: ERROR: [youtube] 9FdHlMOnVjg: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [640, 480], 'bitrate': 496, 'fps': 25.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'strea

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 1271, 'fps': 29.97002997002997, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 1, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 2.62, 'bitrate': 1257, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1920, 1080], 'video_bitrate': 1271, 'video_fps': 2

...

Failed to download https://www.youtube.com/watch?v=1AyT77LqJzQ: ERROR: [youtube] 1AyT77LqJzQ: Video unavailable


ERROR: [youtube] 1AyT77LqJzQ: Video unavailable


Failed to download https://www.youtube.com/watch?v=1AyT77LqJzQ: ERROR: [youtube] 1AyT77LqJzQ: Video unavailable
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 845, 'fps': 29.59, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 1, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 4.04, 'bitrate': 843, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h2

ERROR: [youtube] cJOyCgIKyeA: Video unavailable


Failed to download https://www.youtube.com/watch?v=cJOyCgIKyeA: ERROR: [youtube] cJOyCgIKyeA: Video unavailable


In [22]:
from IPython.display import Video

Video("./msasl_clips/book/BOOK(3)_full.mp4", embed=True, width=640, height=360)

In [26]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_video
import random
from PIL import Image

class VideoDataset(Dataset):
    def __init__(self, root_dir, transform=None, frame_rate=30, num_frames=30):
        """
        Args:
            root_dir (str): Directory with all the videos organized by class labels.
            transform (callable, optional): Optional transform to be applied on a sample.
            frame_rate (int): Sampling rate for frames per second.
            num_frames (int): Number of frames to sample from each video.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.frame_rate = frame_rate
        self.num_frames = num_frames
        
        # Get all subdirectories (i.e., labels)
        self.classes = sorted(os.listdir(root_dir))
        self.video_paths = []
        
        for label in self.classes:
            label_dir = os.path.join(root_dir, label)
            if os.path.isdir(label_dir):
                for file in os.listdir(label_dir):
                    if file.endswith(".mp4"):
                        self.video_paths.append((os.path.join(label_dir, file), label))
        
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path, label = self.video_paths[idx]
        
        # Read the video using torchvision's read_video (returns frames, audio, sample rate)
        frames, _, _ = read_video(video_path)
        
        # Sample the frames if the video is longer than num_frames
        if len(frames) > self.num_frames:
            step = len(frames) // self.num_frames
            frames = frames[::step][:self.num_frames]
        
        # Convert frames to PIL images and apply transforms
        frames = [Image.fromarray(frame.numpy()) for frame in frames]
        
        if self.transform:
            frames = [self.transform(frame) for frame in frames]
        
        # Stack frames to make it (num_frames, channels, height, width)
        frames = torch.stack(frames)
        
        # Convert label to integer (e.g., 'hello' -> 0)
        label_idx = self.classes.index(label)
        
        return frames, label_idx

In [27]:
video_transforms = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize frames to a fixed size
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for RGB
])

In [28]:
# Path to your root video directory
root_dir = "msasl_clips"

# Initialize Dataset and DataLoader
train_dataset = VideoDataset(root_dir=root_dir, transform=video_transforms)

train_loader = DataLoader(
    train_dataset,
    batch_size=16,  # Adjust based on GPU memory
    shuffle=True,
    num_workers=4,  # Adjust for your machine
)

# Check the first batch
for frames, labels in train_loader:
    print(f"Frames shape: {frames.shape}")
    print(f"Labels: {labels}")
    break

/home/segus-faultise/Projects/CAB420_GROUP_A/cab420_GROUP_A_ENV/lib/python3.12/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


ImportError: Caught ImportError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/segus-faultise/Projects/CAB420_GROUP_A/cab420_GROUP_A_ENV/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/segus-faultise/Projects/CAB420_GROUP_A/cab420_GROUP_A_ENV/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipykernel_130179/1708372495.py", line 42, in __getitem__
    frames, _, _ = read_video(video_path)
                   ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/segus-faultise/Projects/CAB420_GROUP_A/cab420_GROUP_A_ENV/lib/python3.12/site-packages/torchvision/io/video.py", line 321, in read_video
    _check_av_available()
  File "/home/segus-faultise/Projects/CAB420_GROU